### Build a Question Answering application over a Graph Database

In [ ]:
pip install neo4j

# pass: "b56aBjU6ibTVgNkGqlR-GJHh-0ds5hozsR0ZoEaZUEY"

Note: you may need to restart the kernel to use updated packages.


In [20]:
from neo4j import GraphDatabase

# Your Aura DB credentials
URI = "neo4j+s://4fc82005.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = "b56aBjU6ibTVgNkGqlR-GJHh-0ds5hozsR0ZoEaZUEY"
DATABASE = "neo4j"  # Default DB for Aura

# Connect and test
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Verify connectivity
try:
    driver.verify_connectivity()
    print("✅ Connection successful!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
finally:
    driver.close()


✅ Connection successful!


In [ ]:
# URI = "neo4j+s://4fc82005.databases.neo4j.io"
# USERNAME = "neo4j"
# PASSWORD = "b56aBjU6ibTVgNkGqlR-GJHh-0ds5hozsR0ZoEaZUEY"

In [21]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [22]:
moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n\n"

In [23]:
driver.execute_query(moview_query)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8568\2527543291.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  driver.execute_query(moview_query)


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000002B657D84EC0>, keys=[])

In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [26]:
groq_api_key

'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'

In [27]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B659976BA0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B659977770>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [41]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=driver,llm=llm,verbose=True)
chain

Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Failed to write data to connection IPv4Address(('si-4fc82005-ce2b.production-orch-0068.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))


AttributeError: 'Neo4jDriver' object has no attribute 'get_structured_schema'

In [13]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Casino"})<-[:DIRECTED]-(p:Person)
RETURN p.name
Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': 'Martin Scorsese.  \n'}

In [14]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.  \n'}

In [17]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)
RETURN count(p) AS artistCount
Full Context:
[{'artistCount': 1240}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1240 artists. \n'}

In [18]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN count(m) 

Full Context:
[{'count(m)': 3}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in', 'result': '3 \n'}